In [24]:
import pandas as pd
import numpy as np
import io
import statsmodels.formula.api as smf
import scipy.stats as stats
import re

In [25]:
date_string = '''totwrk                       -0.169***         
                              (0.018)          
                                               
age                           2.689*           
                              (1.469)          
                                               
south                        101.568**         
                             (41.837)          
                                               
male                         87.669**          
                             (35.104)          
                                               
smsa                         -54.748*          
                             (33.123)          
                                               
yngkid                        -13.962          
                             (50.341)          
                                               
marr                          31.211           
                             (42.233)          
                                               
Constant                    3450.913***        
                             (80.726)   '''
date_string

'totwrk                       -0.169***         \n                              (0.018)          \n                                               \nage                           2.689*           \n                              (1.469)          \n                                               \nsouth                        101.568**         \n                             (41.837)          \n                                               \nmale                         87.669**          \n                             (35.104)          \n                                               \nsmsa                         -54.748*          \n                             (33.123)          \n                                               \nyngkid                        -13.962          \n                             (50.341)          \n                                               \nmarr                          31.211           \n                             (42.233)          \n                   

In [26]:
date_string.strip().split()

['totwrk',
 '-0.169***',
 '(0.018)',
 'age',
 '2.689*',
 '(1.469)',
 'south',
 '101.568**',
 '(41.837)',
 'male',
 '87.669**',
 '(35.104)',
 'smsa',
 '-54.748*',
 '(33.123)',
 'yngkid',
 '-13.962',
 '(50.341)',
 'marr',
 '31.211',
 '(42.233)',
 'Constant',
 '3450.913***',
 '(80.726)']

In [27]:
np.reshape(date_string.strip().split(), (8,3))

array([['totwrk', '-0.169***', '(0.018)'],
       ['age', '2.689*', '(1.469)'],
       ['south', '101.568**', '(41.837)'],
       ['male', '87.669**', '(35.104)'],
       ['smsa', '-54.748*', '(33.123)'],
       ['yngkid', '-13.962', '(50.341)'],
       ['marr', '31.211', '(42.233)'],
       ['Constant', '3450.913***', '(80.726)']], dtype='<U11')

In [30]:
df = pd.DataFrame(np.reshape(date_string.strip().split(), (8,3)), columns=['name', 'estimate', 'std_err'] )

In [31]:
df

,name,estimate,std_err
0,totwrk,-0.169***,(0.018)
1,age,2.689*,(1.469)
2,south,101.568**,(41.837)
3,male,87.669**,(35.104)
4,smsa,-54.748*,(33.123)
5,yngkid,-13.962,(50.341)
6,marr,31.211,(42.233)
7,Constant,3450.913***,(80.726)


- \d -- любая цифра.
- \D -- любая не цифра.
- \w -- любая буква.
- \W -- любая не буква.
- \s -- любой пробельный символ, включая пробел
- \S -- все символы, кроме пробельных
- \* -- любое количество вхождений предыдущего символа(группы символов), от 0 до бесконечности(группы символов)
- \+ -- любое количество вхождений предыдущего символа(группы символов), от 1 до бесконечности(группы символов)
- ? -- 0 или 1 вхождение предыдущего символа(группы символов)
- {n} -- точное количество вхождений - n раз предыдущего символа (группы символов)
- {n,m} -- количество вхождений не менее n и не более m раз предыдущего символа
- | -- логическое "ИЛИ"
- [\,] -- применяется "ИЛИ" к набору символов, любой из которых может встретиться в тексте

###### [-+]?\d+.\d+--- шаблон, который выделяет из строки десятичную дробь
- [-+] -- знак плюс или мину
- ? -- знаков может быть один, может не быть вообще
- \d+ --- неопределенное количество цифр.
- \. --- знак точки, для разделения десятичной дроби

In [35]:
df['num_est'] = df['estimate'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df['num_str_edd'] = df['std_err'].str.extract(r'([-+]?\d+.\d+)').astype(float)

In [36]:
df

,name,estimate,std_err,num_est,num_str_edd
0,totwrk,-0.169***,(0.018),-0.169,0.018
1,age,2.689*,(1.469),2.689,1.469
2,south,101.568**,(41.837),101.568,41.837
3,male,87.669**,(35.104),87.669,35.104
4,smsa,-54.748*,(33.123),-54.748,33.123
5,yngkid,-13.962,(50.341),-13.962,50.341
6,marr,31.211,(42.233),31.211,42.233
7,Constant,3450.913***,(80.726),3450.913,80.726


# $$
P(\hat{\beta_1} - t_{cr}\cdot s_1 < \beta < \hat{\beta_1} + t_{cr} \cdot s_1) = \gamma
$$

In [38]:
alpha=0.1
t_cr = stats.t.ppf(1-alpha/2, 706-8)
t_cr

1.6470395952054095

In [40]:
df['lowe'] = (df['num_est'] - t_cr*df['num_str_edd']).round(2)
df['upper'] = (df['num_est'] + t_cr*df['num_str_edd']).round(2)
df

,name,estimate,std_err,num_est,num_str_edd,lowe,upper
0,totwrk,-0.169***,(0.018),-0.169,0.018,-0.20,-0.14
1,age,2.689*,(1.469),2.689,1.469,0.27,5.11
2,south,101.568**,(41.837),101.568,41.837,32.66,170.48
3,male,87.669**,(35.104),87.669,35.104,29.85,145.49
4,smsa,-54.748*,(33.123),-54.748,33.123,-109.30,-0.19
5,yngkid,-13.962,(50.341),-13.962,50.341,-96.88,68.95
6,marr,31.211,(42.233),31.211,42.233,-38.35,100.77
7,Constant,3450.913***,(80.726),3450.913,80.726,3317.95,3583.87
